# Example reading in a few datasets, and saving and Athena project file

In this example, we read in a few datasets from "raw beamline" datasets, convert to mu(E), 
and save these datasets to an Athena Project file


In [1]:
# first some imports 
from pathlib import Path

from larch.io import read_ascii, AthenaProject
from larch.xafs import pre_edge

from larch.plot.bokeh_xafsplots import plot_mu

In [2]:
# next we will read in a few "raw beamline data files" and create a list of datasets

folder = Path('..', 'xafsdata', 'fexafs')
filenames = ['fexanes_a.001', 'fexanes_b.001', 'fexanes_c.001', 'fexanes_d.001', 'fexanes_e.001']

datasets = []
for filename in filenames:
    # note that these files will have a lot of columns -- we only need "sum_fe_ka"

    dset = read_ascii(Path(folder, filename),  labels='energy, t, i0, i1, i2, sum_out, sum_caka, sum_feka')
    dset.mu = dset.sum_feka / dset.i0
    
    # this works ok for pre-edge subtraction and normalization for this dataset
    pre_edge(dset, pre1=-200.00, pre2=-25.00, nnorm=1, norm1=100.00, norm2=300.00)
    datasets.append(dset)

In [3]:
# now we can plot these mu(E) spectra. 
# With Plotly, we first make a plot, use 'show=False', and keepin passing in that `fig` to re-use the same plot
fig = None
for dset in datasets:
    fig = plot_mu(dset, show_norm=True, label=dset.filename, emin=7050, emax=7250, fig=fig, show=False)
fig.show(xlabel='Energy (eV)', ylabel='Fe XANES', title='Fe XANES spectra')


Loading BokehJS ...

In [4]:
# finally, we can create and Athena Project File and add groups to it, and then save the project
project = AthenaProject(filename=Path(folder, 'FeXANES_example.prj'))
for dset in datasets:
    project.add_group(dset)
project.save(use_gzip=True)